In [1]:
import numpy as np
from keras.datasets import mnist
from keras.utils import to_categorical

In [2]:
def relu(x):
    return np.maximum(0,x)

def relu_prime(x):
    return np.where(x > 0, 1, 0)
    
def mse(y_true, y_pred):
    return np.mean(np.power(y_true-y_pred, 2));

def mse_prime(y_true, y_pred):
    return 2*(y_pred-y_true)/y_true.size;

##### Abstract Class Layer

In [3]:
class Layer:
    def __init__(self):
        self.input = None
        self.output = None

    def forward_propagation(self, input):
        raise NotImplementedError

    def backward_propagation(self, output_error, learning_rate):
        raise NotImplementedError

In [4]:
class DropoutLayer:
    def __init__(self, p):
        self.p = p
        self.mask = None

    def forward_propagation(self, input_data, training=True):
        if training:
            self.mask = np.random.binomial(1, 1 - self.p, size=input_data.shape)
            return input_data * self.mask
        else:
            return input_data * (1 - self.p)

    def backward_propagation(self, output_error, learning_rate):
        return output_error * self.mask

##### Fully-connected Layer

In [5]:
class FCLayer(Layer):
    def __init__(self, input_size, output_size):
        self.weights = np.random.rand(input_size, output_size) - 0.5
        self.bias = np.random.rand(1, output_size) - 0.5

    def forward_propagation(self, input_data):
        self.input = input_data
        self.output = np.dot(self.input, self.weights) + self.bias
        return self.output

    def backward_propagation(self, output_error, learning_rate):
        input_error = np.dot(output_error, self.weights.T)
        weights_error = np.dot(self.input.T, output_error)
        self.weights -= learning_rate * weights_error
        self.bias -= learning_rate * output_error
        return input_error

##### Activation Layer

In [6]:
class ActivationLayer(Layer):
    def __init__(self, activation, activation_prime):
        self.activation = activation
        self.activation_prime = activation_prime

    def forward_propagation(self, input_data):
        self.input = input_data
        self.output = self.activation(self.input)
        return self.output

    def backward_propagation(self, output_error, learning_rate):
        return self.activation_prime(self.input) * output_error

##### Network Class

In [7]:
class Network:
    def __init__(self):
        self.layers = []
        self.loss = None
        self.loss_prime = None

    def add(self, layer):
        self.layers.append(layer)

    def use(self, loss, loss_prime):
        self.loss = loss
        self.loss_prime = loss_prime

    def predict(self, input_data):
        samples = len(input_data)
        result = []
        for i in range(samples):
            output = input_data[i]
            for layer in self.layers:
                output = layer.forward_propagation(output)
            result.append(output)
        return result

    def fit(self, x_train, y_train, epochs, learning_rate):
        samples = len(x_train)
        for i in range(epochs):
            err = 0
            for j in range(samples):
                output = x_train[j]
                for layer in self.layers:
                    output = layer.forward_propagation(output)
                err += self.loss(y_train[j], output)
                error = self.loss_prime(y_train[j], output)
                for layer in reversed(self.layers):
                    error = layer.backward_propagation(error, learning_rate)
            err /= samples
            print('epoch %d/%d   error=%f' % (i+1, epochs, err))

In [8]:
x_train = np.array([[[0,0]], [[0,1]], [[1,0]], [[1,1]]])
y_train = np.array([[[0]], [[1]], [[1]], [[0]]])
np.random.seed(10)
net = Network()
net.add(FCLayer(2, 10))
net.add(ActivationLayer(relu, relu_prime))
net.add(FCLayer(10, 3))
net.add(ActivationLayer(relu, relu_prime))
net.add(DropoutLayer(p=0.1))
net.use(mse, mse_prime)
net.fit(x_train, y_train, epochs=500, learning_rate=0.1)
out = net.predict(x_train)
print(out)

epoch 1/500   error=0.488223
epoch 2/500   error=0.378883
epoch 3/500   error=0.362354
epoch 4/500   error=0.300484
epoch 5/500   error=0.277071
epoch 6/500   error=0.235262
epoch 7/500   error=0.304098
epoch 8/500   error=0.236013
epoch 9/500   error=0.230800
epoch 10/500   error=0.214444
epoch 11/500   error=0.211600
epoch 12/500   error=0.202078
epoch 13/500   error=0.278674
epoch 14/500   error=0.181075
epoch 15/500   error=0.189528
epoch 16/500   error=0.266392
epoch 17/500   error=0.299207
epoch 18/500   error=0.329030
epoch 19/500   error=0.246769
epoch 20/500   error=0.152339
epoch 21/500   error=0.134549
epoch 22/500   error=0.121664
epoch 23/500   error=0.101764
epoch 24/500   error=0.169460
epoch 25/500   error=0.202608
epoch 26/500   error=0.073945
epoch 27/500   error=0.062917
epoch 28/500   error=0.119483
epoch 29/500   error=0.028680
epoch 30/500   error=0.022093
epoch 31/500   error=0.089191
epoch 32/500   error=0.007331
epoch 33/500   error=0.002451
epoch 34/500   erro

In [9]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.reshape(x_train.shape[0], 1, 28*28)
x_train = x_train.astype('float32')
x_train /= 255
y_train = to_categorical(y_train)
x_test = x_test.reshape(x_test.shape[0], 1, 28*28)
x_test = x_test.astype('float32')
x_test /= 255
y_test = to_categorical(y_test)
np.random.seed(10)
net = Network()
net.add(FCLayer(28*28, 100))
net.add(ActivationLayer(relu, relu_prime))
net.add(FCLayer(100, 50))
net.add(ActivationLayer(relu, relu_prime))
net.add(FCLayer(50, 10))
net.add(ActivationLayer(relu, relu_prime))
net.use(mse, mse_prime)
net.fit(x_train, y_train, epochs=20, learning_rate=0.1)
out = net.predict(x_test[0:10])
print("\n")
print("predicted values : ")
print(out, end="\n")
print("true values : ")
print(y_test[0:10])

KeyboardInterrupt: 